In [ ]:
import os
from openai import OpenAI
from PIL import Image
import base64
import json
import pandas as pd
import numpy as np
from collections import Counter
import datetime
import pymssql
from threading import Thread
import functools
import time

# provide your OPENAI API key below
os.environ["OPENAI_API_KEY"] = ""

# This is for connecting to the MSSQL server via pymssql. Adjust it according to your configuration.
conn = pymssql.connect(host=r"(local)", database='test', charset='utf8')

# This is the table name that you will create in the MSSQL server to save the GPT responses.
# Adjust it according to your preferences.
newtablename='20240621_test3'

def most_frequent(numbers):
    # Count the frequency of each number in the list
    frequency = Counter(numbers)
    # Find the number with the highest frequency
    most_common = frequency.most_common(1)
    return most_common[0][0] if most_common else None

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
system_content='''You are a Korean medical school student taking the Korean Medical Licensing Examinations to become a Korean doctor. 
Answer the questions. Give the answer in the following JSON format
{
  "reasoning": "(elaboarate your reasoning here)",
  "answer": 4
}

'''

client = OpenAI()

df=pd.read_excel("C:/Users/woong/Downloads/analysis/2021.xlsx")
df['Question'][0] + df["Followings"][0]


# this is the "timeout" decorator function. 
def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

sql_createtable="CREATE TABLE [" + newtablename +"""] 
(
    questionnum    NVARCHAR(20),
    GPT_reasoning    NVARCHAR(max),
    GPT_answer    NVARCHAR(max),
    Answer    NVARCHAR(20),
)

"""
conn = pymssql.connect(host=r"(local)", database='test', charset='utf8')
with conn:
    with conn.cursor() as cur:
        cur.execute(sql_createtable)
        conn.commit()

        
        
# After approximately estimating your tokens per prompt,
# calculate how many seconds each iteration can run without exceeding the TPM limit 
# (I would recommend not exceeding 70~80% of the TPM limit)
# Then, set the corresponding seconds in the "timeout" decorator function below.

@timeout(0.5)
def insert_into_sql(i, system_content, model_ver, numbers, temperature_setting):

    question=df['Question'][i] + df["Followings"][i]
    answer=df['Answer'][i]
    
    
    if pd.isna(df['image1'][i]):
        response = client.chat.completions.create(
            model=model_ver,
            n=numbers,
            temperature=temperature_setting,
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": [
                    {"type": "text", "text": question},
                ]}
            ],
        )
    elif pd.notna(df['image3'][i]):
        image1=encode_image("C:/Users/woong/Downloads/analysis/images/"+df['image1'][i])
        image2=encode_image("C:/Users/woong/Downloads/analysis/images/"+df['image2'][i])
        image3=encode_image("C:/Users/woong/Downloads/analysis/images/"+df['image3'][i])
        response = client.chat.completions.create(
            model=model_ver,
            n=numbers,
            temperature=temperature_setting,
            response_format={ "type": "json_object" },
            messages=[{"role": "system", "content": system_content},
                      {"role": "user", "content": [
                          {"type": "text", "text": question},
                          {"type": "image_url", "image_url": {
                              "url": f"data:image/png;base64,{image1}"}
                                },
                          {"type": "image_url", "image_url": {
                              "url": f"data:image/png;base64,{image2}"}
                                },
                          {"type": "image_url", "image_url": {
                              "url": f"data:image/png;base64,{image3}"}
                                }
                            ]}
                        ],
                    )
    
    elif pd.notna(df['image2'][i]):
        image1=encode_image("C:/Users/woong/Downloads/analysis/images/"+df['image1'][i])
        image2=encode_image("C:/Users/woong/Downloads/analysis/images/"+df['image2'][i])
        response = client.chat.completions.create(
            model=model_ver,
            n=numbers,
            temperature=temperature_setting,
            response_format={ "type": "json_object" },
            messages=[{"role": "system", "content": system_content},
                      {"role": "user", "content": [
                          {"type": "text", "text": question},
                          {"type": "image_url", "image_url": {
                              "url": f"data:image/png;base64,{image1}"}
                                },
                          {"type": "image_url", "image_url": {
                              "url": f"data:image/png;base64,{image2}"}
                                }
                            ]}
                        ],
                    )        
    elif pd.notna(df['image1'][i]):
        image1=encode_image("C:/Users/woong/Downloads/analysis/images/"+df['image1'][i])
        response = client.chat.completions.create(
            model=model_ver,
            n=numbers,
            temperature=temperature_setting,
            response_format={ "type": "json_object" },
            messages=[{"role": "system", "content": system_content},
                      {"role": "user", "content": [
                          {"type": "text", "text": question},
                          {"type": "image_url", "image_url": {
                              "url": f"data:image/png;base64,{image1}"}
                                }
                            ]}
                        ],
                    )        
    
    aa2=response.choices
    
    for j in range(len(aa2)):
        data_dict = json.loads(aa2[j].message.content)
        try:
            reasoning_temp=data_dict['reasoning']
            answer_temp=data_dict['answer']
            
            conn = pymssql.connect(host=r"(local)", database='test', charset='utf8')
            sql = "INSERT INTO [" + newtablename+"] (questionnum, GPT_reasoning, GPT_answer, Answer) VALUES (%s, %s, %s, %s)"

            with conn:
                with conn.cursor() as cur:
                    cur.execute(sql, (str(df['class'][i])+"_"+str(df['Q_no'][i]),str(reasoning_temp),str(answer_temp), str(df['Answer'][i])))
                    conn.commit()
        except:
            pass


In [ ]:
for i in range(len(df)):
    insert_into_sql(i, system_content, 'gpt-4o', 20, 1.0)